In [11]:
import requests

API_KEY = '67570a1e-eb29-4ef1-b214-40b159dada45'
url = "https://api.harvardartmuseums.org/classification"
params = {
    "apikey": API_KEY,
    "classification": "Paintings",  
    "size": 100,  
    "page": 1
}

response = requests.get(url, params)
data = response.json()

print(data['records'])  

[{'objectcount': 1, 'name': 'Natural History Specimens', 'id': 1133, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 1133}, {'objectcount': 1, 'name': 'Paintings with Text', 'id': 197, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 197}, {'objectcount': 66, 'name': 'Performance Artifacts', 'id': 224, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 224}, {'objectcount': 470, 'name': 'Rubbings', 'id': 173, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 173}, {'objectcount': 62, 'name': 'Brick Stamps', 'id': 304, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 304}, {'objectcount': 46, 'name': 'Cameos', 'id': 1086, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 1086}, {'objectcount': 1, 'name': 'Casts', 'id': 1139, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid': 1139}, {'objectcount': 6, 'name': 'Containers', 'id': 1203, 'lastupdate': '2025-08-20T04:39:22-04:00', 'classificationid':

In [12]:
data.keys()  

dict_keys(['info', 'records'])

In [13]:
record=data['records']
for record in data['records']:
    if record['objectcount']>=2500:
        print(record['name'])

Accessories (non-art)
Photographs
Drawings
Prints
Paintings
Sculpture
Coins
Vessels
Textile Arts
Archival Material
Fragments
Manuscripts
Seals
Straus Materials


In [14]:
import mysql.connector


mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",


)
mycursor = mydb.cursor()


In [ ]:
import requests
import mysql.connector

API_KEY = "67570a1e-eb29-4ef1-b214-40b159dada45"
CLASSIFICATION = "Drawings"
RECORDS_TO_FETCH = 2500
RECORDS_PER_PAGE = 100

def create_tables():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="harvard_artifacts"
    )
    cursor = mydb.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS artifact_metadata (
            id INT PRIMARY KEY,
            title TEXT,
            culture TEXT,
            period TEXT,
            century TEXT,
            medium TEXT,
            dimensions TEXT,
            description TEXT,
            department TEXT,
            classification TEXT,
            accessionyear INT,
            accessionmethod TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS artifact_media (
            objectid INT PRIMARY KEY,
            imagecount INT,
            mediacount INT,
            colorcount INT,
            rank INT,
            datebegin INT,
            dateend INT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS artifact_colors (
            objectid INT,
            color VARCHAR(100),
            spectrum VARCHAR(100),
            hue VARCHAR(100),
            percent FLOAT,
            css3 VARCHAR(100),
            PRIMARY KEY (objectid, color)
        )
    """)
    mydb.commit()
    mydb.close()

def fetch_artifact_data():
    all_records = []
    for page in range(1, RECORDS_TO_FETCH // RECORDS_PER_PAGE + 1):
        response = requests.get("https://api.harvardartmuseums.org/object", params={
            "apikey": API_KEY,
            "classification": CLASSIFICATION,
            "size": RECORDS_PER_PAGE,
            "page": page
        })
        data = response.json()
        all_records.extend(data.get("records", []))
    return all_records

def extract_and_store(records):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="harvard_artifacts"
    )
    cursor = mydb.cursor()

    for obj in records:
        cursor.execute("""
            INSERT IGNORE INTO artifact_metadata (
                id, title, culture, period, century, medium, dimensions,
                description, department, classification, accessionyear, accessionmethod
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            obj.get("id"),
            obj.get("title"),
            obj.get("culture"),
            obj.get("period"),
            obj.get("century"),
            obj.get("medium"),
            obj.get("dimensions"),
            obj.get("description"),
            obj.get("department"),
            obj.get("classification"),
            obj.get("accessionyear"),
            obj.get("accessionmethod")
        ))

        cursor.execute("""
            INSERT IGNORE INTO artifact_media (
                objectid, imagecount, mediacount, colorcount, rank, datebegin, dateend
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            obj.get("id"),
            obj.get("imagecount"),
            obj.get("mediacount"),
            obj.get("colorcount"),
            obj.get("rank"),
            obj.get("datebegin"),
            obj.get("dateend")
        ))

        for color in obj.get("colors", []):
            cursor.execute("""
                INSERT IGNORE INTO artifact_colors (
                    objectid, color, spectrum, hue, percent, css3
                ) VALUES (%s, %s, %s, %s, %s, %s)
            """, (
                obj.get("id"),
                color.get("color"),
                color.get("spectrum"),
                color.get("hue"),
                color.get("percent"),
                color.get("css3")
            ))

    mydb.commit()
    mydb.close()

create_tables()
records = fetch_artifact_data()
extract_and_store(records)


In [ ]:
!pip install streamlit

In [16]:
import streamlit as st
import pandas as pd
import mysql.connector
import requests

# DB CONNECTION
def get_connection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  
        database="harvard_artifacts"
    )

# Run SQL Query
def run_query(query):
    conn = get_connection()
    df = pd.read_sql(query, conn)
    conn.close()
    return df

# HARVARD API FETCH
API_KEY = "67570a1e-eb29-4ef1-b214-40b159dada45"
BASE_URL = "https://api.harvardartmuseums.org/object"

def fetch_artifacts(classification, rows=2500):
    params = {
        "apikey": API_KEY,
        "classification": classification,
        "size": rows
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    return data.get("records", [])

# STREAMLIT APP
st.set_page_config(page_title="🏛 Harvard Artifacts Collection", layout="wide")
st.title("🎨🏛 Harvard’s Artifacts Collection")

# Tabs for workflow
tab1, tab2, tab3 = st.tabs(["📥 Collect Data", "🗄 Migrate to SQL", "📊 SQL Queries"])

# TAB 1: COLLECT DATA
with tab1:
    st.subheader("Fetch Records from Harvard API")
    
    # Multi-classification selector
    classifications = st.multiselect(
        "Choose one or more classifications:",
        ["Coins", "Photographs", "Drawings", "Vessels", "Paintings"],
        default=["Coins"]
    )

    if st.button("Collect Data"):
        all_records = []
        for classification in classifications:
            records = fetch_artifacts(classification)
            if records:
                st.success(f"✅ Collected {len(records)} records for {classification}")
                
                # Show a preview for each classification
                with st.expander(f"Preview: {classification}"):
                    st.json(records[0])  # metadata preview
                    st.json(records[0].get("images", []))  # media preview
                    st.json(records[0].get("colors", []))  # colors preview
                
                all_records.extend(records)
            else:
                st.error(f"❌ No records found for {classification}")
        
        if all_records:
            st.write(f"📦 Total Records Collected: {len(all_records)}")

# TAB 2: MIGRATE TO SQL
with tab2:
    st.subheader("Insert the collected data into MySQL")
    if st.button("Insert"):
        # TODO: Transform + Insert records into SQL tables
        st.success("✅ Data Inserted Successfully")
        
        # Example preview from DB
        try:
            df = run_query("SELECT * FROM artifact_metadata LIMIT 5;")
            st.write("Inserted Data Preview (artifact_metadata):")
            st.dataframe(df)
        except Exception as e:
            st.error(f"Error fetching data: {e}")

# TAB 3: SQL QUERIES
queries = {
    # ---------------- artifact_metadata ----------------
    "Artifacts from 11th century Byzantine culture":
        "SELECT * FROM artifact_metadata WHERE century = '11th century' AND culture = 'Byzantine';",

    "Unique cultures represented":
        "SELECT DISTINCT culture FROM artifact_metadata;",

    "Artifacts from Archaic Period":
        "SELECT * FROM artifact_metadata WHERE period = 'Archaic';",

    "Artifact titles by accession year (DESC)":
        "SELECT title, accessionyear FROM artifact_metadata ORDER BY accessionyear DESC;",

    "Artifacts count per department":
        "SELECT department, COUNT(*) AS artifact_count FROM artifact_metadata GROUP BY department;",

    # ---------------- artifact_media ----------------
    "Artifacts with more than 1 image":
        "SELECT objectid, COUNT(*) AS image_count FROM artifact_media GROUP BY objectid HAVING COUNT(*) > 1;",

    "Average rank of all artifacts":
        "SELECT AVG(rank) AS avg_rank FROM artifact_media;",

    "Artifacts with higher colorcount than mediacount":
        "SELECT m.objectid FROM artifact_media m JOIN artifact_colors c ON m.objectid = c.objectid GROUP BY m.objectid HAVING COUNT(c.hue) > COUNT(m.mediaid);",

    "Artifacts created between 1500 and 1600":
        "SELECT * FROM artifact_metadata WHERE dated BETWEEN 1500 AND 1600;",

    "Artifacts with no media files":
        "SELECT * FROM artifact_metadata WHERE objectid NOT IN (SELECT DISTINCT objectid FROM artifact_media);",

    # ---------------- artifact_colors ----------------
    "Distinct hues used":
        "SELECT DISTINCT hue FROM artifact_colors;",

    "Top 5 most used colors":
        "SELECT hue, COUNT(*) AS usage_count FROM artifact_colors GROUP BY hue ORDER BY usage_count DESC LIMIT 5;",

    "Average coverage percentage per hue":
        "SELECT hue, AVG(percentage) AS avg_coverage FROM artifact_colors GROUP BY hue;",

    "Colors for a given artifact ID (example: 19971)":
        "SELECT hue, percentage FROM artifact_colors WHERE objectid = 19971;",

    "Total number of color entries":
        "SELECT COUNT(*) AS total_colors FROM artifact_colors;",

    # ---------------- Join-Based ----------------
    "Titles and hues for Byzantine culture":
        "SELECT m.title, c.hue FROM artifact_metadata m JOIN artifact_colors c ON m.objectid = c.objectid WHERE m.culture = 'Byzantine';",

    "Artifact titles with associated hues":
        "SELECT m.title, GROUP_CONCAT(c.hue) AS hues FROM artifact_metadata m JOIN artifact_colors c ON m.objectid = c.objectid GROUP BY m.title;",

    "Titles, cultures, and media ranks where period is not null":
        "SELECT m.title, m.culture, me.rank FROM artifact_metadata m JOIN artifact_media me ON m.objectid = me.objectid WHERE m.period IS NOT NULL;",

    "Top 10 ranked artifacts with hue Grey":
        "SELECT m.title, me.rank FROM artifact_metadata m JOIN artifact_media me ON m.objectid = me.objectid JOIN artifact_colors c ON m.objectid = c.objectid WHERE c.hue = 'Grey' ORDER BY me.rank ASC LIMIT 10;",

    "Artifacts per classification with avg media count":
        """SELECT m.classification, COUNT(DISTINCT m.objectid) AS artifact_count,
                  AVG(COUNT(me.mediaid)) OVER (PARTITION BY m.classification) AS avg_media_count
           FROM artifact_metadata m
           LEFT JOIN artifact_media me ON m.objectid = me.objectid
           GROUP BY m.classification;"""
}

with tab3:
    st.subheader("Explore with Pre-written SQL Queries")
    selected_query = st.selectbox("Choose a Query", list(queries.keys()))
    if st.button("Run Query"):
        try:
            df = run_query(queries[selected_query])
            st.dataframe(df, use_container_width=True)
            
            # Auto-chart if aggregated
            if "COUNT" in queries[selected_query] or "AVG" in queries[selected_query]:
                try:
                    st.bar_chart(df.set_index(df.columns[0]))
                except:
                    pass
        except Exception as e:
            st.error(f"Error running query: {e}")


2025-08-20 16:28:18.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.800 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-20 16:28:18.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar